In [10]:
from yolov5.detect import run as detect_with_yolov5

# Step 1: Object Detection with YOLOv5
def detect_objects_yolov5(image_path):
    # Run YOLOv5 detection
    results = detect_with_yolov5(weights='yolov5s.pt', source=image_path, exist_ok=True)
    
    # Example usage
image_path = 'C:\\Users\\sony\\yolov5\\data\\images\\image-0.jpg'
detected_objects = detect_objects_yolov5(image_path)


YOLOv5  2024-4-6 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/1 C:\Users\sony\yolov5\data\images\image-0.jpg: 448x640 1 bench, 1 handbag, 21 suitcases, 573.0ms
Speed: 0.0ms pre-process, 573.0ms inference, 2.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp


In [6]:
from torchvision import models
import torch
from PIL import Image
from torchvision import transforms

# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Load and preprocess the image
img = Image.open('C:\\Users\\sony\\yolov5\\data\\images\\image-0.jpg')
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

# Perform inference
alexnet.eval()
out = alexnet(batch_t)

# Load class labels
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
print("Number of classes: {}".format(len(classes)))

# Get top predictions
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
top_predictions = [(classes[idx], percentage[idx].item()) for idx in indices[0][:10]]
print("Top predictions:", top_predictions)


Number of classes: 1000
Top predictions: [('crate', 39.634193420410156), ('carton', 16.762165069580078), ('bakery, bakeshop, bakehouse', 13.19906234741211), ('gas pump, gasoline pump, petrol pump, island dispenser', 5.520036220550537), ('bookshop, bookstore, bookstall', 4.00698709487915), ('forklift', 1.8441518545150757), ('hamper', 1.5510622262954712), ('tobacco shop, tobacconist shop, tobacconist', 0.9393933415412903), ('grocery store, grocery, food market, market', 0.9313933849334717), ('refrigerator, icebox', 0.9113894701004028)]


In [4]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")

        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    pixel_values = pixel_values.to(device)

    output_ids = model.generate(pixel_values, **gen_kwargs)

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

# Example usage
predictions = predict_step(['C:\\Users\\sony\\yolov5\\data\\images\\image-0.jpg'])
print(predictions)


C:\Users\sony\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


C:\Users\sony\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


['a pile of luggage sitting on the side of a street']


In [1]:
from yolov5.detect import run as detect_with_yolov5
from torchvision import models, transforms
from PIL import Image
import torch
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer

# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Load class labels
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

# Load pre-trained ViT-GPT2 image captioning model
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def detect_classify_caption(image_path):
    # Step 1: Object Detection with YOLOv5
    results = detect_with_yolov5(weights='yolov5s.pt', source=image_path, exist_ok=True)
    
    # Step 2: Image Classification with AlexNet
    img = Image.open(image_path)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    alexnet.eval()
    out = alexnet(batch_t)
    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    top_predictions = [(classes[idx], percentage[idx].item()) for idx in indices[0][:10]]
    
    # Step 3: Image Captioning with ViT-GPT2
    pixel_values = feature_extractor(images=[img], return_tensors="pt").pixel_values.to(device)
    max_length = 16
    num_beams = 4
    gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
    output_ids = model.generate(pixel_values, **gen_kwargs)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return results, top_predictions, preds

# Example usage
image_path = 'C:\\Users\\sony\\yolov5\\data\\images\\image-0.jpg'
object_detection_results, classification_results, caption_predictions = detect_classify_caption(image_path)
print("Object Detection Results:", object_detection_results)
print("Image Classification Results:", classification_results)
print("Image Captioning Predictions:", caption_predictions)


YOLOv5  2024-4-6 Python-3.11.5 torch-2.2.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/1 C:\Users\sony\yolov5\data\images\image-0.jpg: 448x640 1 bench, 1 handbag, 21 suitcases, 840.5ms
Speed: 5.0ms pre-process, 840.5ms inference, 34.8ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\exp
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Object Detection Results: None
Image Classification Results: [('crate', 39.634193420410156), ('carton', 16.762165069580078), ('bakery, bakeshop, bakehouse', 13.19906234741211), ('gas pump, gasoline pump, petrol pump, island dispenser', 5.520036220550537), ('bookshop, bookstore, bookstall', 4.00698709487915), ('forklift', 1.8441518545150757), ('hamper', 1.5510622262954712), ('tobacco shop, tobacconist shop, tobacconist', 0.9393933415412903), ('grocery store, grocery, food market, market', 0.9313933849334717), ('refrigerator, icebox', 0.9113894701004028)]
Image Captioning Predictions: ['a pile of luggage sitting on the side of a street']
